# Load from Features Notebook

In [1]:
import dill
dill.load_session("Notebook_Saves/features.db")

In [2]:
import h2o
from h2o.automl import H2OAutoML
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_121"; OpenJDK Runtime Environment (Zulu 8.20.0.5-linux64) (build 1.8.0_121-b15); OpenJDK 64-Bit Server VM (Zulu 8.20.0.5-linux64) (build 25.121-b15, mixed mode)
  Starting server from /home/ubuntu/anaconda3/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpua7isukh
  JVM stdout: /tmp/tmpua7isukh/h2o_ubuntu_started_from_python.out
  JVM stderr: /tmp/tmpua7isukh/h2o_ubuntu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.2
H2O cluster version age:,1 month and 1 day
H2O cluster name:,H2O_from_python_ubuntu_h4hqd2
H2O cluster total nodes:,1
H2O cluster free memory:,6.981 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
training_df = h2o.H2OFrame(training)
validation_df = h2o.H2OFrame(submit)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [28]:
x = training_df.drop(['user_id_hash', 'user_purchase_binary_7_days',
                           'user_purchase_binary_14_days']).columns

y = 'user_purchase_binary_7_days'

## AutoML

In [29]:
aml = H2OAutoML(balance_classes=True, max_models = 10, seed = 8, keep_cross_validation_predictions=True, sort_metric='auc')

In [30]:
aml.train(x=x, y=y, training_frame=training_df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [32]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_1_AutoML_20190220_223801,0.994023,0.0145983,0.186055,0.0651701,0.00424714
StackedEnsemble_BestOfFamily_AutoML_20190220_223801,0.993083,0.0211997,0.18439,0.0689763,0.00475773
StackedEnsemble_AllModels_AutoML_20190220_223801,0.993083,0.0211997,0.18439,0.0689763,0.00475773
XRT_1_AutoML_20190220_223801,0.988045,0.0369704,0.183767,0.085469,0.00730495
DRF_1_AutoML_20190220_223801,0.986757,0.0298047,0.198643,0.0817812,0.00668817


In [33]:
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20190220_223801


ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.0038477692940338793
RMSE: 0.062030390084489066
LogLoss: 0.013274472388990806
Mean Per-Class Error: 0.025637567528291383
AUC: 0.9953529090753119
pr_auc: 0.6935078925865752
Gini: 0.9907058181506239
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.33164718250433606: 


,False,True,Error,Rate
False,612797.0,1897.0,0.0031,(1897.0/614694.0)
True,1622.0,3107.0,0.343,(1622.0/4729.0)
Total,614419.0,5004.0,0.0057,(3519.0/619423.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3316472,0.6384465,188.0
max f2,0.1498819,0.7067930,266.0
max f0point5,0.5283387,0.6760421,121.0
max accuracy,0.4697620,0.9948468,140.0
max precision,0.9672965,1.0,0.0
max recall,0.0035961,1.0,392.0
max specificity,0.9672965,1.0,0.0
max absolute_mcc,0.3316472,0.6358437,188.0
max min_per_class_accuracy,0.0204103,0.9699725,361.0
max mean_per_class_accuracy,0.0099380,0.9743624,378.0


Gains/Lift Table: Avg response rate:  0.76 %, avg score:  0.76 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
,2019-02-20 23:18:57,20 min 20.785 sec,0.0,0.5,0.6931472,0.5,0.0,1.0,0.9923655
,2019-02-20 23:19:02,20 min 25.657 sec,5.0,0.3898711,0.4939786,0.9920074,0.5843221,66.0694950,0.0065642
,2019-02-20 23:19:05,20 min 28.564 sec,10.0,0.3058299,0.3643980,0.9922161,0.5937574,66.6019978,0.0065270
,2019-02-20 23:19:08,20 min 31.767 sec,15.0,0.2413692,0.2745211,0.9926769,0.6021166,66.9402936,0.0066029
,2019-02-20 23:19:11,20 min 34.986 sec,20.0,0.1922577,0.2101023,0.9931187,0.6108095,67.5323114,0.0062897
---,---,---,---,---,---,---,---,---,---
,2019-02-20 23:20:45,22 min 9.073 sec,245.0,0.0622686,0.0133725,0.9952798,0.6892067,72.6701798,0.0058829
,2019-02-20 23:20:47,22 min 10.715 sec,250.0,0.0622075,0.0133459,0.9952954,0.6895746,72.6490363,0.0057457
,2019-02-20 23:20:49,22 min 12.767 sec,255.0,0.0621315,0.0133186,0.9953154,0.6917241,72.7124668,0.0057973
,2019-02-20 23:20:52,22 min 15.402 sec,260.0,0.0620614,0.0132879,0.9953416,0.6928212,72.8181842,0.0055180



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
session_gap,122090.0546875,1.0,0.4820897
purchase_count,38778.4335938,0.3176216,0.1531221
spend,29852.2265625,0.2445099,0.1178757
event_gap,19294.6875000,0.1580365,0.0761878
previous_sessions_duration,14834.2060547,0.1215022,0.0585749
---,---,---,---
country.FI,4.0218000,0.0000329,0.0000159
country.NG,3.9200499,0.0000321,0.0000155
country.NO,2.9919400,0.0000245,0.0000118
country.AT,2.6804199,0.0000220,0.0000106



See the whole table with table.as_data_frame()


In [34]:
h2o.save_model(aml.leader, path = "Models/")

'/home/ubuntu/leanplum/final-project-team-worldwide/Models/XGBoost_1_AutoML_20190220_223801'

In [38]:
seven_day_pred = aml.leader.predict(validation_df.drop(['user_id_hash', 'user_purchase_binary_7_days',
                              'user_purchase_binary_14_days']))

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [56]:
submission['user_purchase_binary_7_days'] = seven_day_pred['True'].as_data_frame()

In [57]:
x = training_df.drop(['user_id_hash', 'user_purchase_binary_7_days',
                           'user_purchase_binary_14_days']).columns

y = 'user_purchase_binary_14_days'

In [59]:
aml = H2OAutoML(balance_classes=True, max_models = 10, seed = 8, sort_metric='auc')

In [60]:
aml.train(x=x, y=y, training_frame=training_df)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [61]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
XGBoost_1_AutoML_20190220_223801,0.994023,0.0145983,0.186055,0.0651701,0.00424714
StackedEnsemble_BestOfFamily_AutoML_20190220_223801,0.993083,0.0211997,0.18439,0.0689763,0.00475773
StackedEnsemble_AllModels_AutoML_20190220_223801,0.993083,0.0211997,0.18439,0.0689763,0.00475773
XRT_1_AutoML_20190220_223801,0.988045,0.0369704,0.183767,0.085469,0.00730495
DRF_1_AutoML_20190220_223801,0.986757,0.0298047,0.198643,0.0817812,0.00668817
XGBoost_1_AutoML_20190220_235848,0.98387,0.0212671,0.185136,0.0747618,0.00558933
DRF_1_AutoML_20190220_235848,0.968439,0.0418701,0.211495,0.0917946,0.00842625
XRT_1_AutoML_20190220_235848,0.963414,0.0494418,0.206461,0.0961029,0.00923576


In [62]:
aml.leader

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_1_AutoML_20190220_223801


ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.0038477692940338793
RMSE: 0.062030390084489066
LogLoss: 0.013274472388990806
Mean Per-Class Error: 0.025637567528291383
AUC: 0.9953529090753119
pr_auc: 0.6935078925865752
Gini: 0.9907058181506239
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.33164718250433606: 


,False,True,Error,Rate
False,612797.0,1897.0,0.0031,(1897.0/614694.0)
True,1622.0,3107.0,0.343,(1622.0/4729.0)
Total,614419.0,5004.0,0.0057,(3519.0/619423.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.3316472,0.6384465,188.0
max f2,0.1498819,0.7067930,266.0
max f0point5,0.5283387,0.6760421,121.0
max accuracy,0.4697620,0.9948468,140.0
max precision,0.9672965,1.0,0.0
max recall,0.0035961,1.0,392.0
max specificity,0.9672965,1.0,0.0
max absolute_mcc,0.3316472,0.6358437,188.0
max min_per_class_accuracy,0.0204103,0.9699725,361.0
max mean_per_class_accuracy,0.0099380,0.9743624,378.0


Gains/Lift Table: Avg response rate:  0.76 %, avg score:  0.76 %



...

Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
,2019-02-20 23:18:57,20 min 20.785 sec,0.0,0.5,0.6931472,0.5,0.0,1.0,0.9923655
,2019-02-20 23:19:02,20 min 25.657 sec,5.0,0.3898711,0.4939786,0.9920074,0.5843221,66.0694950,0.0065642
,2019-02-20 23:19:05,20 min 28.564 sec,10.0,0.3058299,0.3643980,0.9922161,0.5937574,66.6019978,0.0065270
,2019-02-20 23:19:08,20 min 31.767 sec,15.0,0.2413692,0.2745211,0.9926769,0.6021166,66.9402936,0.0066029
,2019-02-20 23:19:11,20 min 34.986 sec,20.0,0.1922577,0.2101023,0.9931187,0.6108095,67.5323114,0.0062897
---,---,---,---,---,---,---,---,---,---
,2019-02-20 23:20:45,22 min 9.073 sec,245.0,0.0622686,0.0133725,0.9952798,0.6892067,72.6701798,0.0058829
,2019-02-20 23:20:47,22 min 10.715 sec,250.0,0.0622075,0.0133459,0.9952954,0.6895746,72.6490363,0.0057457
,2019-02-20 23:20:49,22 min 12.767 sec,255.0,0.0621315,0.0133186,0.9953154,0.6917241,72.7124668,0.0057973
,2019-02-20 23:20:52,22 min 15.402 sec,260.0,0.0620614,0.0132879,0.9953416,0.6928212,72.8181842,0.0055180



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
session_gap,122090.0546875,1.0,0.4820897
purchase_count,38778.4335938,0.3176216,0.1531221
spend,29852.2265625,0.2445099,0.1178757
event_gap,19294.6875000,0.1580365,0.0761878
previous_sessions_duration,14834.2060547,0.1215022,0.0585749
---,---,---,---
country.FI,4.0218000,0.0000329,0.0000159
country.NG,3.9200499,0.0000321,0.0000155
country.NO,2.9919400,0.0000245,0.0000118
country.AT,2.6804199,0.0000220,0.0000106



See the whole table with table.as_data_frame()


In [64]:
h2o.save_model(aml.leader, path = ".")

'/home/ubuntu/leanplum/final-project-team-worldwide/XGBoost_1_AutoML_20190220_223801'

In [65]:
fourteen_day_pred = aml.leader.predict(validation_df.drop(['user_id_hash', 'user_purchase_binary_7_days',
                              'user_purchase_binary_14_days']))

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [66]:
submission['user_purchase_binary_14_days'] = fourteen_day_pred['True'].as_data_frame()

In [67]:
submission.to_csv('submissionH2O.csv', index=False)